<a href="https://colab.research.google.com/github/fazallm/shopee-ndsc/blob/master/ndsc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link1 ='https://drive.google.com/open?id=1AAKixj0o6DL24zObdHuFDSayLrQRmBNR'

In [0]:
fluff, id = link1.split('=')
print (id)

1AAKixj0o6DL24zObdHuFDSayLrQRmBNR


In [0]:
downloaded1 = drive.CreateFile({'id':id}) 
downloaded1.GetContentFile('train.csv')  
df1 = pd.read_csv('train.csv')

In [0]:
df1.head()

,itemid,title,Category,image_path
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,461203,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg


In [0]:
link2='https://drive.google.com/open?id=1IlxOxmLaKaDKkIbdF7Mijk5v15x2TeMZ'
fluff,id2= link2.split('=')
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('test.csv')
df2=pd.read_csv('test.csv')

In [0]:
def extract(link, filename):
  fluff,id= link.split('=')
  downloaded = drive.CreateFile({'id': id})
  downloaded.GetContentFile(filename)
  df=pd.read_csv(filename)
  return df

In [0]:
title=df1['title'].tolist()
category=df1['Category'].tolist()
tes=df2['title'].tolist()

In [0]:
tet=[]
for i in title:
    tet.append(i)
for i in tes:
    tet.append(i)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector=TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None)
all_title=vector.fit_transform(tet)

In [0]:
train_title=all_title[0:len(category)]
tests_title=all_title[len(category):]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_title, category, test_size=0.25, random_state=40)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
parameters = {'solver':('newton-cg', 'lbfgs', 'sag')}
logs=LogisticRegression(random_state=20, multi_class='multinomial')
log_reg=GridSearchCV(logs, parameters, cv=5)
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=20, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'solver': ('newton-cg', 'lbfgs', 'sag')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [0]:
y_pred=log_reg.predict(X_test)
from sklearn.metrics import accuracy_score
print ('testing acc is %f' %accuracy_score(y_pred, y_test))

testing acc is 0.717655


In [0]:
pred=log_reg.predict(tests_title)

In [0]:
subm=extract('https://drive.google.com/open?id=1rvNIHhGKHSxemOxg5IJcW7LEJKwaFEX5', 'data_info_val_sample_submission.csv')

In [0]:
subm['Category']=pred
subm.to_csv('Submission_naive bayes.csv', index=False)